In [1]:
import src.partner_api2_pb2_grpc as api
from dateutil.relativedelta import relativedelta  
from src.partner_api2_pb2 import *
import pandas as pd
import numpy as np
import grpc
from datetime import datetime
import time, glob
from sqlalchemy import create_engine  

In [2]:
    current_date = datetime.fromtimestamp(time.time()).timestamp()
    partnerApiEndpoint = 'partner.emporiaenergy.com:50052'
    creds = grpc.ssl_channel_credentials()
    channel = grpc.secure_channel(partnerApiEndpoint, creds)
    stub = api.PartnerApiStub(channel)
    request = AuthenticationRequest()
    request.partner_email = 'hsoto@cigepty.com'
    request.password = 'hsm280466'
    auth_response = stub.Authenticate(request=request)
    auth_token = auth_response.auth_token
    inventoryRequest = DeviceInventoryRequest()
    inventoryRequest.auth_token = auth_token
    inventoryResponse = stub.GetDevices(inventoryRequest)
    deviceUsageRequest = DeviceUsageRequest()
    deviceUsageRequest.auth_token = auth_token
    deviceUsageRequest.start_epoch_seconds = int(current_date)
    deviceUsageRequest.end_epoch_seconds = int(current_date)
    deviceUsageRequest.scale = DataResolution.Days
    deviceUsageRequest.channels = DeviceUsageRequest.UsageChannel.ALL
    usageResponse = stub.GetUsageData(deviceUsageRequest)

In [7]:
from src.db_verify import checking_data

device_name, data = checking_data('techo')
lista_separada = [valor.split('-') for valor in data.columns]
fases = [elemento[1] for elemento in lista_separada if len(elemento) == 2]
etiquetas = [elemento[1] for elemento in lista_separada if len(elemento) >= 3]
circuito = [elemento[2] for elemento in lista_separada if len(elemento) >= 3]
data.columns = ['Time Bucket'] + fases + circuito
data_ = data.copy()
data_.columns = ['Time Bucket'] + fases + etiquetas
data_ = data_.groupby(data_.columns, axis=1).sum()
data = data.merge(data_, on='Time Bucket')
data = data[[col for col in data.columns if not col.endswith('_y')]]
data = data.rename(columns=lambda col: col.replace('_x', ''))
data['Consumo total'] = data[fases].sum(axis=1)

In [8]:
import plotly.express as px
import datetime, calendar
import pandas as pd

fig1 = px.bar(x=data["Time Bucket"], y=data["Consumo total"])
fig1.update_traces(marker_color='#668616')
fig1.update_layout(title="Consumo total por día", yaxis_title="Consumo (kWh)", xaxis_title="",
                        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False), 
                        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')